# Classification of degenerate tetrahedra with rational dihedral angles

The purpose of this notebook is to give a computational classification of configurations of four vectors $\mathbf{v}_1, \mathbf{v}_2, \mathbf{v}_3, \mathbf{v}_4 \in \mathbb{R}^3$, no two collinear,
with $\mathbf{v}_2, \mathbf{v}_3, \mathbf{v}_4$ coplanar and $\mathbf{v}_1$ not in the same plane, 
for which for every $j,k$, the angle $\theta_{jk}$ between $\mathbf{v}_j$ and $\mathbf{v}_k$ is a rational multiple of $\pi$.

Taking $x_{jk} = e^{i \theta_{jk}}$, this corresponds to finding solutions of the *Gram determinant equation*
$$
\det(M) = 0, \qquad M = \begin{pmatrix} 2 & x_{12} + x_{12}^{-1} & x_{13} + x_{13}^{-1} & x_{14} + x_{14}^{-1} \\
x_{12} + x_{12}^{-1} & 2 & x_{23} + x_{23}^{-1} & x_{24} + x_{24}^{-1} \\
x_{13} + x_{13}^{-1} & x_{23} + x_{23}^{-1} & 2 & x_{34} + x_{34}^{-1} \\
x_{14} + x_{14}^{-1} & x_{24} + x_{24}^{-1} & x_{34} + x_{34}^{-1} & 2
\end{pmatrix}
$$
in roots of unity in which none of the variables equals $\pm 1$, we have
$$
x_{23} x_{24} x_{34} = 1
$$
(which forces the bottom right $3 \times 3$ minor to vanish), and none of the other $3 \times 3$ diagonal minors vanishes.

We consider solutions up to equivalence by the action of the group generated by:
- the action of $S_3$ on $\{2,3,4\}$;
- the action of $(\mathbb{Z}/2\mathbb{Z})^3$ given by $x_{1k} \mapsto x_{1k}^{-1}$;
- the involution $x_{12}, x_{23}, x_{24} \mapsto -x_{12}, -x_{23}, -x_{24}$;
- the involution $x_{12}, x_{23}, x_{24} \mapsto x_{12}^{-1}, x_{23}^{-1}, x_{24}^{-1}$.

We also consider the enhanced group generated by these actions plus:
- the modified Regge symmetry: 
$$
(x_{12}, x_{34}, x_{13}, x_{24}, x_{14}, x_{23}) \mapsto (x_{12}, x_{34}, s x_{23}^{-1}, s x_{14}^{-1}, s x_{24}^{-1}, s x_{13}^{-1}) \qquad (s = \sqrt{x_{13} x_{14} x_{23} x_{24}}).
$$
We find that:
- there are 3 orbits of two-parameter solutions, which form a single orbit under the enhanced group;
- there are 13 orbits of one-parameter solutions, which form 3 orbits under the enhanced group;
- every isolated solution is either an isolated solution of the equation $\det(M) \equiv 0 \pmod{2}$ or is equivalent to a solution with values in $\mathbb{Q}(\zeta_N)$ where $N \in \{168, 180, 240\}$.

This code was originally run using SageMath version 9.2. Allow roughly 3 hours on a single CPU to complete all computations.

## Global imports and declarations

Import some utility functions from the Python `itertools` module.

In [1]:
from itertools import chain, combinations, permutations, product, repeat

Import a module to compute Hermite normal forms of integer matrices.

In [2]:
import sage.matrix.matrix_integer_dense_hnf as hnf

## Aside: factorization of the Gram equation

We compute the Gram equation and check that under the additional condition $x_{23} x_{24} x_{34} = 1$, $\det(M)$ has a square root which is a polynomial in the $x_{ij}$. This can also be explained using the spherical law of cosines. (This is for illustration only; no commutative algebra is used in the rest of the computation.)

In [3]:
P.<x_12,x_34,x_13,x_24,x_14,x_23> = LaurentPolynomialRing(QQ, 6, order='degrevlex')

In [4]:
# Note: in Sage, ~x is a shorthand for x^-1.
gram_matrix = Matrix([[2, x_12+~x_12, x_13+~x_13, x_14+~x_14],
                       [x_12+~x_12, 2, x_23+~x_23, x_24+~x_24],
                       [x_13+~x_13, x_23+~x_23, 2, x_34+~x_34],
                       [x_14+~x_14, x_24+~x_24, x_34+~x_34, 2]])
print(gram_matrix)

[             2 x_12 + x_12^-1 x_13 + x_13^-1 x_14 + x_14^-1]
[x_12 + x_12^-1              2 x_23 + x_23^-1 x_24 + x_24^-1]
[x_13 + x_13^-1 x_23 + x_23^-1              2 x_34 + x_34^-1]
[x_14 + x_14^-1 x_24 + x_24^-1 x_34 + x_34^-1              2]


In [5]:
gram_determinant = gram_matrix.det()

In [6]:
gram_det_reduced = sqrt(gram_determinant.subs({x_23: 1/(x_24*x_34)}))
print(gram_det_reduced)

-x_34*x_24*x_14 + x_12*x_34 + x_13*x_24 - x_34*x_24*x_14^-1 - x_13*x_24^-1 - x_12*x_34^-1 + x_12^-1*x_34 + x_13^-1*x_24 + x_34^-1*x_24^-1*x_14 - x_13^-1*x_24^-1 - x_12^-1*x_34^-1 + x_34^-1*x_24^-1*x_14^-1


## Matrices of angle relations (matrels)

We express a parametric solution of the Gram determinant equation as an augmented matrix $M$ in which each row vector $(a_1,\dots,a_6,b)$ with $a_1,\dots,a_6 \in \mathbb{Z}$ and $b \in \mathbb{Q}$ expresses a relation of the form
\[
a_1 \alpha_1 + \cdots + a_6 \alpha_6 \equiv b \pmod{\mathbb{Z}}.
\]
We refer to such a matrix as a *matrel*.

We say a matrel is *regularized* if the left subdivision is in Hermite normal form and the right subdivision has entries in $[0,1) \cap \QQ$,
and *resolved* if in addition the pivots of the left subdivision are all equal to 1.

## Function declarations: matrices of angle relations

Regularize a matrel, by applying Sage's HNF computation to the left subdivision.

In [7]:
def regularize_matrel(M, w=6):
    M1 = copy(M)
    M1.subdivide(col_lines=w)
    A = M1.subdivision(0,0).change_ring(ZZ)
    _, U = hnf.hnf_with_transformation(A)
    M1 = U*M1
    for i in range(M1.dimensions()[0]):
        M1[i,w] = frac(M1[i,w])
    return M1

Resolve a matrel, yielding its resolutions one at a time.

In [8]:
def resolve_matrel(M, w=6):
    M1 = regularize_matrel(M, w=w)
    # Check for inconsistencies
    n = M1.dimensions()[0]
    while n > 0 and M1.subdivision(0,0).row(n-1) == 0:
        if M1.subdivision(0,1)[n-1,0] != 0:
            # This matrel is inconsistent, return nothing.
            return
        M1 = M1.delete_rows([n-1])
        # The previous operation does not preserve the subdivision; restore it now.
        M1.subdivide(col_lines=w)
        n -= 1
    queue = [M1]
    while queue:
        M2 = queue.pop(0)
        M2 = regularize_matrel(M2, w=w)
        for i in range(n):
            d = gcd([Integer(x) for x in M2.row(i)[:w]])
            if d == 1:
                continue
            # We need to divide this row by d. This creates d options
            # for the last entry, which we add back to the queue.
            for j in range(d):
                M3 = copy(M2)
                M3[i, w] += j
                M3.rescale_row(i, 1/d)
                queue.append(M3)
            break
        else:
            # If we survive this far, then there are no factors left to remove;
            # hence we yield this matrix.
            M2.set_immutable()
            yield M2

Test whether one matrel implies (is a specialization of) a second. The matrel `M1` must be regularized and resolved.

This is done by regularizing the concatenation of the two matrices and testing whether the rank equals the rank of `M1` (otherwise it must increase).

In [9]:
def test_implication_matrel(M1, M2):
    n1 = M1.dimensions()[0]
    n2 = M2.dimensions()[0]
    M = regularize_matrel(Matrix(M1.rows() + M2.rows()))
    return all(M.row(i) == 0 for i in range(n1, n1+n2))

Test a (regularized, resolved) matrel for any of the following degeneracy conditions.

- One of the angles equals 0 or $\pi$ (that is, two vectors are equal or opposite).
- One of the other diagonal $3 \times 3$ minors vanishes.
- The vector $\mathbf{v}_1$ is perpendicular to the plane spanned by the other three. (This is a cheat to save time; we will need to reintroduce this matrel later.)

In [10]:
degenerate_matrels = [Matrix([[(1 if j == i else 0) for j in range(7)]]) for i in range(6)] + \
    [Matrix([[(1 if j==i else 1/2 if j==6 else 0) for j in range(7)]]) for i in range(6)] + \
    [Matrix([[1,0,1,0,0,1,0]]), Matrix([[1,0,1,0,0,-1,0]]), Matrix([[1,0,-1,0,0,1,0]]), Matrix([[1,0,-1,0,0,-1,0]]),
     Matrix([[1,0,0,1,1,0,0]]), Matrix([[1,0,0,1,-1,0,0]]), Matrix([[1,0,0,-1,1,0,0]]), Matrix([[1,0,0,-1,-1,0,0]]),
     Matrix([[0,1,1,0,1,0,0]]), Matrix([[0,1,1,0,-1,0,0]]), Matrix([[0,1,-1,0,1,0,0]]), Matrix([[0,1,-1,0,-1,0,0]]),
     Matrix([[1,0,0,0,0,0,1/4], [0,0,1,0,0,0,1/4], [0,0,0,0,1,0,1/4]]),
     Matrix([[1,0,0,0,0,0,1/4], [0,0,1,0,0,0,1/4], [0,0,0,0,1,0,3/4]]),
     Matrix([[1,0,0,0,0,0,1/4], [0,0,1,0,0,0,3/4], [0,0,0,0,1,0,1/4]]),
     Matrix([[1,0,0,0,0,0,1/4], [0,0,1,0,0,0,3/4], [0,0,0,0,1,0,3/4]]),
     Matrix([[1,0,0,0,0,0,3/4], [0,0,1,0,0,0,1/4], [0,0,0,0,1,0,1/4]]),
     Matrix([[1,0,0,0,0,0,3/4], [0,0,1,0,0,0,1/4], [0,0,0,0,1,0,3/4]]),
     Matrix([[1,0,0,0,0,0,3/4], [0,0,1,0,0,0,3/4], [0,0,0,0,1,0,1/4]]),
     Matrix([[1,0,0,0,0,0,3/4], [0,0,1,0,0,0,3/4], [0,0,0,0,1,0,3/4]])]

def is_degenerate_matrel(M):
    return any(test_implication_matrel(M, M1) for M1 in degenerate_matrels)

Pretty-print a matrel by converting it into a parametric solution. If `raw` is True, we return a vector of constant terms, then a vector of coefficients for each of the parameters. If `raw` is False, we return a LaTeX representation.

In [11]:
def render_matrel(M, raw=False):
    dim = 6 - M.rank()
    tmp = all(M[i,M.pivots()[i]] == 1 for i in range(6-dim))
    if not tmp:
        # Swap columns
        M = copy(M)
        M.swap_columns(3, 5)
    free_cols = [i for i in range(6) if not i in M.pivots()]
    M1 = M.augment(Matrix(QQ, 6-dim, dim))
    M1 = Matrix(M1.rows() + [tuple((1 if j in (free_cols[i],7+i) else 0) for j in range(7+dim)) for i in range(dim)])
    if not tmp:
        M1.swap_columns(3, 5)
    M1 = M1.echelon_form()
    if raw:
        M1 = M1.transpose()
        M1 = Matrix(M1.rows()[6:])
        M1.rescale_row(0, 2)
        return M1.rows()
    ans = "("
    for i in range(6):
        s = ""
        if M1[i,6]:
            c = 2*M1[i,6]
            if c == 1:
                s += r'\pi'
            elif c == -1:
                s += r'-\pi'
            else:
                if c.numerator() == 1:
                    numstr = ""
                elif c.numerator() == -1:
                    numstr = "-"
                else:
                    numstr = str(c.numerator())
                s += r'\tfrac{' + numstr + r'\pi}{' + str(c.denominator()) + '}'
        for j in range(dim):
            if M1[i,7+j]:
                if s != "" and M1[i,7+j] > 0:
                    s += "+"
                if M1[i,7+j] == -1:
                    s += "-"
                elif M1[i,7+j] != 1:
                    s += str(M1[i,7+j])
                s += r'\gamma_'+str(j+1)
        if i>0:
            ans += ", "
        ans += s
    return(ans + ")")

## Function declaration: symmetry reduction

Apply a modified Regge symmetry to a matrel.

In [12]:
def apply_regge_matrel(M):
    regge_mat = Matrix(QQ, 
                       [[0,1,0,0,0,0,0,0],
                        [0,0,1,0,0,0,0,0],
                        [1,0,0,0,0,0,-1,0],
                        [1,0,0,0,0,-1,0,0],
                        [1,0,0,0,-1,0,0,0],
                        [1,0,0,-1,0,0,0,0],
                        [0,0,0,0,0,0,0,1]])
    M1 = M*regge_mat
    M1 = Matrix(M1.rows() + [[-2,0,0,1,1,1,1,0]])
    for M2 in resolve_matrel(M1, w=7):
        yield M2.submatrix(1,1)

Given a list (or generator) of matrels, yield orbit representatives for the symmetry group $G$ generated by the $S_3$-action on vectors fixing $v_1$; negation of the vectors; individual negation of the angles abutting $v_1$; and joint negation of the angles not abutting $v_1$. 

This is done by computing connected components of a Cayley graph for some generators of $G$.

To speed things up, we do not keep track of zero-dimensional matrels. In particular, we do not decide whether these solutions are degenerate.

Additional optional arguments:

- `l2`: if set, specifies some additional matrels which should be considered as degeneracy conditions. Any matrel which specializes one of these is to be excluded. (This requires that elements of `l` be resolved.)
- `use_regge`: if True, include a Regge symmetry among the generators.
- `return_orbits`: if True, instead of returning orbit representatives, return intersections of `l` with the orbits.
- `orders`: if not None, assumed to be a list passed by reference. The multiplicative orders of zero-dimensional matrels are appended to this list.

In [13]:
def reduce_by_symmetry(l, l2=None, use_regge=False, return_orbits=False, orders=None):
    M0 = Matrix([[1,0,0,0,0,0,1/2],
                 [0,1,0,0,0,0,0],
                 [0,0,1,0,0,0,0],
                 [0,0,0,1,0,0,1/2],
                 [0,0,0,0,1,0,0],
                 [0,0,0,0,0,1,1/2],
                 [0,0,0,0,0,0,1]])
    queue = list(set(l))
    num_gens = 5 if use_regge else 4
    vertices = []
    edges = []
    while queue:
        M = queue.pop(0)
        # If this matrel is of dimension 0, retain only its field order.
        if M.rank() == 6:
            for M1 in resolve_matrel(M):
                c = lcm(x.denominator() for x in M1.list())
                if not c in orders:
                    orders.append(c)
            continue
        M.set_immutable()
        vertices.append(M)
        # If M is degenerate, treat it as being connected to a dummy vertex labeled 0.
        if l2 and any(test_implication_matrel(M, M1) for M1 in l2):
            edges.append((M, 0))
        for t in ((0,1,2), (0,2,1), (1,2,0)):
            for sg in (range(1,num_gens) if t==(0,1,2) else range(1)):
                M1 = copy(M)
                Mc = M.columns()
                # Action of S_3.
                for i in range(3):
                    for j in range(2):
                        M1.set_column(2*i+j, Mc[2*t[i]+j])
                # Negation of an angle from vector 1.
                if sg == 1:
                    M1.rescale_col(0, -1)
                # Negation of the three angles among vectors 2,3,4.
                if sg == 2:
                    for i in range(3):
                        M1.rescale_col(2*i+1, -1)
                # Negation of vector 1.
                if sg == 3:
                    M1 = M1 * M0
                # Regge symmetry.
                if sg == 4:
                    M1 = list(apply_regge_matrel(M1))[0]
                # Regularize.
                M1 = regularize_matrel(M1)
                M1.set_immutable()
                edges.append((M, M1))
                # If the other endpoint is a new matrel, add it to the queue.
                if M1 not in vertices and M1 not in queue:
                    queue.append(M1)
    Gamma = Graph(edges, loops=True, immutable=True)
    if return_orbits:
        # Yield a tuple of all elements of l in each connected component except the dummy component.
        for l4 in Gamma.connected_components(sort=False):
            if 0 not in l4:
                yield tuple(x for x in l if x in l4)
    else:
        # Yield one representative from each connected component except the dummy component.
        for l4 in Gamma.connected_components(sort=False):
            if 0 not in l4:
                yield l4[0]

## Function declarations: iteration over cosine relations

Enumerate minimal cosine relations modulo permutation, negation, and Galois symmetry.

The classification is taken from Poonen-Rubinstein. We do not actually need the relations of type 5 or 6.

In [14]:
h = 1/2
rel_dict = {
    "1": ((1/4,),),
    "2*": ((0, h),),
    "3": ((1/3+h, 1/5, 2/5),),
    "3*": ((0, 1/3, 2/3),),
    "4": ((1/3+h, 1/3+1/5+h, 2/3+1/5+h, 2/5),
          (1/3+h, 1/3+2/5+h, 2/3+2/5+h, 1/5),
          (1/3+h, 1/7, 2/7, 3/7),),
    "5": ((1/3+h, 1/3+1/7+h, 2/3+1/7+h, 2/7, 3/7),
          (1/3+h, 1/3+2/7+h, 2/3+2/7+h, 1/7, 3/7),
          (1/3+h, 1/3+3/7+h, 2/3+3/7+h, 1/7, 2/7),
          (1/5+h, 2/5+h, 1/7, 2/7, 3/7),),
    "5*": ((0, 1/5, 2/5, 3/5, 4/5),),
    "6": ((1/3+h, 1/3+1/7+h, 1/3+2/7+h, 2/3+1/7+h, 2/3+2/7+h, 3/7),
          (1/3+h, 1/3+1/7+h, 1/3+3/7+h, 2/3+1/7+h, 2/3+3/7+h, 2/7),
          (1/3+h, 1/3+2/7+h, 1/3+3/7+h, 2/3+2/7+h, 2/3+3/7+h, 1/7),
          (1/5+h, 2/5+h, 1/3+1/7+h, 2/3+1/7+h, 2/7, 3/7),
          (1/5+h, 2/5+h, 1/3+2/7+h, 2/3+2/7+h, 1/7, 3/7),
          (1/5+h, 2/5+h, 1/3+3/7+h, 2/3+3/7+h, 1/7, 2/7),
          (2/5+h, 1/3+1/5, 2/3+1/5, 1/7, 2/7, 3/7),
          (1/5+h, 1/3+2/5, 2/3+2/5, 1/7, 2/7, 3/7),
          (1/3+h, 1/11, 2/11, 3/11, 4/11, 5/11),),
    "6*": ((1/3+h, 2/3+h, 1/5, 2/5, 3/5, 4/5),)
}

Check that these are all valid relations.

In [15]:
for i in rel_dict:
    for rel in rel_dict[i]:
        N = lcm(x.denominator() for x in rel)
        K.<zeta> = CyclotomicField(N)
        assert(sum(zeta^(N*j)+zeta^(-N*j) for j in rel) == 0)

Add shifts of $1/2$ to the relations that do not already include an arbitrary shift.

In [16]:
for i in rel_dict:
    if i in ("3", "4", "5", "6"):
        l = list(rel_dict[i])
        l2 = [tuple(x+h for x in j) for j in l]
        rel_dict[i] = tuple(l+l2)

Yield all matrels of a given form, without resolving.

In [17]:
def relations_by_form(form):
    shape = []
    ct = 0
    for s in form:
        s1 = s[:-1] if s[-1] in "*abcd" else s
        s1 = int(s1)
        shape.append(tuple(range(ct, ct+s1)))
        ct += s1
    # The rows of the matrix M1 represent the expression of the beta angles
    # in terms of the original angles.
    M1 = Matrix(QQ,
                [[-1,-1,0,0,0,0,-1/4],
                [1,-1,0,0,0,0,-1/4],
                [0,0,-1,-1,0,0,-1/4],
                [0,0,1,-1,0,0,-1/4],
                [0,0,0,0,-1,-1,-1/4],
                [0,0,0,0,1,-1,-1/4],
                [0,0,0,0,0,0,1]])
    # Choose a cosine relation, and a signed permutation of the beta angles.
    for t in product(*tuple(rel_dict[s] for s in form)):
        for perm in permutations(range(6)):
            for sg in product([-1,1], repeat=6):
                # Impose the degeneration constraint.
                l = [{0: -1/2, 1: -1/2, 2: -1/2, 3: -1/2, 4: -1/2, 5: -1/2, 6: -3/4}]
                # Impose the chosen cosine relation on the beta angles.
                for i, s in enumerate(shape):
                    if "*" not in form[i]:
                        for j in range(len(s)):
                            l.append({perm[s[j]]: sg[s[j]],
                                      6: t[i][j]})
                    else:
                        for (j1,j2) in combinations(range(len(shape[i])), int(2)):
                            l.append({perm[s[j1]]: sg[s[j1]],
                                      perm[s[j2]]: -sg[s[j2]],
                                      6: t[i][j1] - t[i][j2]})
                # Create the matrix M with rows specified by l.
                M = Matrix({(i,j): l[i][j] for i in range(len(l)) for j in l[i]})
                # Convert back to the original angles.
                M = M*M1
                # Convert to an augmented matrix.
                M.subdivide(col_lines=6)
                # Regularize the presentation before returning.
                M = regularize_matrel(M)
                # Make the matrix immutable and hence hashable.
                M.set_immutable()
                yield M

Generate unresolved matrels; divide by symmetry; resolve; and yield those which are nondegenerate. Note that the resolution step means that we must divide by symmetry again later.

In [18]:
def solutions_by_form(form, orders):
    for M in reduce_by_symmetry(relations_by_form(form), orders=orders):
        for M1 in resolve_matrel(M):
            if not is_degenerate_matrel(M1):
                yield M1, 6 - M1.rank()

## Computation of cosine relations by form

Create the list of all possible forms of matrels. We omit forms with more than one 1.

In [19]:
rels = list(rel_dict.keys())
rels.sort(reverse=True)
vecs = WeightedIntegerVectors(6, [int(s[:-1] if "*" in s else s) for s in rels])
form_list = [tuple(chain(*tuple(repeat(rels[i], v[i]) for i in range(len(rels))))) for v in vecs]
form_list = [form for form in form_list if form.count("1") <= 1]
form_list.sort(key = lambda form: sum(1 for s in form if "*" in s), reverse=True)

For all forms with at least one free parameter, generate an exhaustive (but possibly redundant) set of symmetry representatives for (regularized, resolved) matrels of this form; then categorize these by dimension. When the dimension is 0, instead of keeping the relation itself, we retain only its multiplicative order (in order to reconstruct these efficiently later).

In [20]:
d = {1: [], 2: []}
orders = []
for form in form_list:
    if sum(1 for s in form if "*" in s) >= 1:
        print(form)
        for M, dim in solutions_by_form(form, orders):
            d[dim].append(M)

('2*', '2*', '2*')


('3*', '3*')


('3*', '2*', '1')


('6*',)


('5*', '1')


('4', '2*')


('3*', '3')


('3', '2*', '1')


## Analysis of the results

Check that the 2-dimensional matrels form two orbits.

In [21]:
len(d[2])

6

In [22]:
ans2a = list(reduce_by_symmetry(d[2], degenerate_matrels))
assert(len(ans2a)==2)

Add back the third parametric solution with $\mathbf{v}_1$ perpendicular to the plane of $\mathbf{v}_2, \mathbf{v}_3, \mathbf{v}_4$ (which we had previously treated as degenerate for the sake of efficiency).

In [23]:
ans2a.insert(0, Matrix([[1,0,0,0,0,0,1/4], [0,0,1,0,0,0,1/4], [0,0,0,0,1,0,1/4], [0,1,0,1,0,1,0]]))
ans2a[0].set_immutable()

Check that these form a unique orbit when Regge symmetries are included.

In [24]:
ans2b = list(reduce_by_symmetry(ans2a, use_regge=True))
assert(len(ans2b)==1)

Print these matrels as parametric solutions.

In [25]:
for M in ans2a:
    print(render_matrel(M, raw=True))

[(1/2, 0, 1/2, 0, 1/2, 0), (0, -1, 0, 0, 0, 1), (0, -1, 0, 1, 0, 0)]
[(0, -1, 0, 0, 1/2, 1), (1, 0, 1, 0, 0, 0), (0, 1, 0, 1, 0, -2)]
[(0, -1/2, 3/2, 0, 0, 1/2), (-1, -1, 2, 0, 1, 1), (3, 1, -3, 1, 0, -2)]


In [26]:
for M in ans2a:
    print(render_matrel(M))

(\tfrac{\pi}{2}, -\gamma_1-\gamma_2, \tfrac{\pi}{2}, \gamma_2, \tfrac{\pi}{2}, \gamma_1)
(\gamma_1, -\pi+\gamma_2, \gamma_1, \gamma_2, \tfrac{\pi}{2}, \pi-2\gamma_2)
(-\gamma_1+3\gamma_2, \tfrac{-\pi}{2}-\gamma_1+\gamma_2, \tfrac{3\pi}{2}+2\gamma_1-3\gamma_2, \gamma_2, \gamma_1, \tfrac{\pi}{2}+\gamma_1-2\gamma_2)


Check that the 1-dimensional matrels, excluding any specializations of 2-dimensional ones, form thirteen orbits.

In [27]:
ans1a = list(reduce_by_symmetry(d[1], d[2]))
assert(len(ans1a)==13)

Group these into orbits for the enhanced symmetry group.

In [28]:
ans1b = list(reduce_by_symmetry(ans1a, use_regge=True, return_orbits=True))

Print these matrels as parametric solutions.

In [29]:
for t in ans1b:
    for M in t:
        print(render_matrel(M, raw=True))
    print("")

[(1/6, 5/6, 11/6, 7/6, 0, 0), (4, 0, 3, -1, 1, 1)]
[(0, 0, 1/3, 0, 2/3, 0), (2, 2, 1, -3, 3, 1)]
[(1/6, 5/6, 11/6, 7/6, 1/2, 0), (2, 0, 1, -1, 0, 1)]
[(1/6, 5/6, 11/6, 7/6, 1/3, 0), (3, 1, 2, -2, 3, 1)]
[(0, 0, 1/3, 0, 1, 0), (1, 3, 0, -4, 1, 1)]
[(0, 0, 1/3, 0, 1/2, 0), (1, 1, 0, -2, 0, 1)]

[(1/6, 5/6, 11/6, 7/6, 1/3, 0), (3, 0, 2, -1, 0, 1)]
[(1/6, 0, 1/2, 0, 5/6, 0), (2, 1, 1, -2, 2, 1)]
[(1/6, 2/3, 11/6, 4/3, 1/6, 0), (2, 1, 1, -2, 2, 1)]
[(0, 0, 1/3, 0, 2/3, 0), (1, 2, 0, -3, 0, 1)]

[(1/6, 5/6, 11/6, 7/6, 0, 0), (5, 1, 2, -2, 5, 1)]
[(0, 0, 1/3, 0, 1, 0), (3, 3, 0, -4, 5, 1)]
[(1/3, 0, 1/2, 0, 1, 0), (0, 2, 0, -3, 1, 1)]



In [30]:
for t in ans1b:
    for M in t:
        print(render_matrel(M), r"\\")
    print(r"\hline")

(\tfrac{\pi}{6}+4\gamma_1, \tfrac{5\pi}{6}, \tfrac{11\pi}{6}+3\gamma_1, \tfrac{7\pi}{6}-\gamma_1, \gamma_1, \gamma_1) \\
(2\gamma_1, 2\gamma_1, \tfrac{\pi}{3}+\gamma_1, -3\gamma_1, \tfrac{2\pi}{3}+3\gamma_1, \gamma_1) \\
(\tfrac{\pi}{6}+2\gamma_1, \tfrac{5\pi}{6}, \tfrac{11\pi}{6}+\gamma_1, \tfrac{7\pi}{6}-\gamma_1, \tfrac{\pi}{2}, \gamma_1) \\
(\tfrac{\pi}{6}+3\gamma_1, \tfrac{5\pi}{6}+\gamma_1, \tfrac{11\pi}{6}+2\gamma_1, \tfrac{7\pi}{6}-2\gamma_1, \tfrac{\pi}{3}+3\gamma_1, \gamma_1) \\
(\gamma_1, 3\gamma_1, \tfrac{\pi}{3}, -4\gamma_1, \pi+\gamma_1, \gamma_1) \\
(\gamma_1, \gamma_1, \tfrac{\pi}{3}, -2\gamma_1, \tfrac{\pi}{2}, \gamma_1) \\
\hline
(\tfrac{\pi}{6}+3\gamma_1, \tfrac{5\pi}{6}, \tfrac{11\pi}{6}+2\gamma_1, \tfrac{7\pi}{6}-\gamma_1, \tfrac{\pi}{3}, \gamma_1) \\
(\tfrac{\pi}{6}+2\gamma_1, \gamma_1, \tfrac{\pi}{2}+\gamma_1, -2\gamma_1, \tfrac{5\pi}{6}+2\gamma_1, \gamma_1) \\
(\tfrac{\pi}{6}+2\gamma_1, \tfrac{2\pi}{3}+\gamma_1, \tfrac{11\pi}{6}+\gamma_1, \tfrac{4\pi}{3}-2\gamma

List the multiplicative orders of 0-dimensional matrels, and verify that each one divides a value of $N$ in our original list.

In [31]:
sorted(set(orders))

[5, 10, 15, 20, 30, 60, 84, 120, 168, 180, 240]

In [32]:
orders_bound = set([168, 180, 240])
assert all(any(y%x==0 for y in orders_bound) for x in orders)

This completes the proof.